In [42]:
import os
import cv2
import numpy as np
import imageio
from PIL import Image

In [43]:
folder_paths = [
    "Generated Images - 100 IDs/Generated_Images_100_10/ID_0000000",
    "Generated Images - 100 IDs/Generated_Images_100_10/ID_0000001",
    "Generated Images - 100 IDs/Generated_Images_100_10/ID_0000002",
    "Generated Images - 100 IDs/Generated_Images_100_10/ID_0000003",
    "Generated Images - 100 IDs/Generated_Images_100_10/ID_0000004",
    "Generated Images - 100 IDs/Generated_Images_100_10/ID_0000005",
    "Generated Images - 100 IDs/Generated_Images_100_10/ID_0000006",
    "Generated Images - 100 IDs/Generated_Images_100_10/ID_0000007"
]

In [44]:
output_dir = "Visualizing_Generated_Images"
os.makedirs(output_dir, exist_ok=True)

In [45]:
image_lists = []
grid_size = (8, 8)

In [46]:
max_images_per_folder = 8
for folder_path in folder_paths:
    images = []
    count = 0
    for filename in os.listdir(folder_path):
        if count >= max_images_per_folder:
            break
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            images.append(image)
            count += 1
    image_lists.append(images)

In [47]:
image_width, image_height = image_lists[0][0].size

In [48]:
grid_image = Image.new('RGB', (image_width * grid_size[1], image_height * grid_size[0]))

In [49]:
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        folder_index = j
        image_index = i
        if folder_index < len(image_lists) and image_index < len(image_lists[folder_index]):
            grid_image.paste(image_lists[folder_index][image_index], (j * image_width, i * image_height))

In [50]:
grid_image.save(os.path.join(output_dir, "1.Combined_image.png"))

In [51]:
def draw_keypoints(image, keypoints):
    img_with_keypoints = image.copy()
    for kp in keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        # Idea: Drawing a blue circle for each key-pint identified
        cv2.circle(img_with_keypoints, (x, y), 3, (255, 0, 0), -1)
    return img_with_keypoints

In [52]:
def process_fingerprint_image(image_path):
    fingerprint_img = cv2.imread(image_path)
    sift = cv2.SIFT_create()
    keypoints = sift.detect(fingerprint_img, None)
    image_with_keypoints = draw_keypoints(fingerprint_img, keypoints)
    return image_with_keypoints

In [53]:
processed_images = []

In [54]:
for folder_path in folder_paths:
    images = []
    count = 0
    for filename in os.listdir(folder_path):
        if count >= max_images_per_folder:
            break
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            image_with_keypoints = process_fingerprint_image(image_path)
            images.append(image_with_keypoints)
            count += 1
    processed_images.append(images)

In [55]:
image_width, image_height = processed_images[0][0].shape[1], processed_images[0][0].shape[0]
grid_image = Image.new('RGB', (image_width * grid_size[1], image_height * grid_size[0]))

In [56]:
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        folder_index = j
        image_index = i
        if folder_index < len(processed_images) and image_index < len(processed_images[folder_index]):
            img = processed_images[folder_index][image_index]
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            grid_image.paste(img, (j * image_width, i * image_height))

In [57]:
grid_image.save(os.path.join(output_dir, "2.Combined_image_with_keypoints.png"))

In [58]:
def process_fingerprint_image(image_path):
    fingerprint_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(fingerprint_img, (5, 5), 0)

    # Perform Canny edge detection
    edges = cv2.Canny(blurred, 30, 150)

    # Perform Hough transform for line detection
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)

    # If lines are detected, visualize them on the original image
    if lines is not None:
        lines_img = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        for line in lines:
            rho, theta = line[0]
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(lines_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    else:
        lines_img = np.zeros_like(fingerprint_img)

    return edges, lines_img

In [59]:
edge_detection_results = []
lines_visualized_images = []

In [60]:
for folder_path in folder_paths:
    edge_detection_results_per_folder = []
    lines_visualized_images_per_folder = []
    count = 0
    for filename in os.listdir(folder_path):
        if count >= max_images_per_folder:
            break
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            edges, lines_img = process_fingerprint_image(image_path)
            edge_detection_results_per_folder.append(edges)
            lines_visualized_images_per_folder.append(lines_img)
            count += 1
    edge_detection_results.append(edge_detection_results_per_folder)
    lines_visualized_images.append(lines_visualized_images_per_folder)

In [61]:
image_width, image_height = edge_detection_results[0][0].shape[1], edge_detection_results[0][0].shape[0]
grid_image_edges = Image.new('RGB', (image_width * grid_size[1], image_height * grid_size[0]))

In [62]:
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        folder_index = j
        image_index = i
        if folder_index < len(edge_detection_results) and image_index < len(edge_detection_results[folder_index]):
            img = edge_detection_results[folder_index][image_index]
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            img = Image.fromarray(img)
            grid_image_edges.paste(img, (j * image_width, i * image_height))

In [63]:
grid_image_edges.save(os.path.join(output_dir, "3.Combined_edge_detection_results.png"))

In [64]:
grid_image_lines = Image.new('RGB', (image_width * grid_size[1], image_height * grid_size[0]))

for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        folder_index = j
        image_index = i
        if folder_index < len(lines_visualized_images) and image_index < len(lines_visualized_images[folder_index]):
            img = lines_visualized_images[folder_index][image_index]
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            grid_image_lines.paste(img, (j * image_width, i * image_height))

In [65]:
grid_image_lines.save(os.path.join(output_dir, "4.Combined_lines_visualized_images.png"))

In [66]:
def process_fingerprint_image(image_path):
    fingerprint_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(fingerprint_img, (5, 5), 0)

    # Applying Sobel operator for edge detection
    sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = np.uint8(magnitude)

    # Threshold the edge magnitude to get binary image
    _, binary = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Applying morphological operations to enhance the ridges
    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # Finding contours to identify fingerprint regions
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the original image
    contours_img = cv2.cvtColor(fingerprint_img, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(contours_img, contours, -1, (0, 0, 255), 2)

    # Convert the images to PIL format
    edge_detection_result = Image.fromarray(magnitude)
    binary_image = Image.fromarray(binary)
    enhanced_binary_image = Image.fromarray(binary)
    contours_image = Image.fromarray(cv2.cvtColor(contours_img, cv2.COLOR_BGR2RGB))

    return edge_detection_result, binary_image, enhanced_binary_image, contours_image

In [67]:
edge_detection_results = []
binary_images = []
enhanced_binary_images = []
contours_images = []

In [68]:
for folder_path in folder_paths:
    edge_detection_results_per_folder = []
    binary_images_per_folder = []
    enhanced_binary_images_per_folder = []
    contours_images_per_folder = []
    count = 0
    for filename in os.listdir(folder_path):
        if count >= max_images_per_folder:
            break
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            edge_detection_result, binary_image, enhanced_binary_image, contours_image = process_fingerprint_image(image_path)
            edge_detection_results_per_folder.append(edge_detection_result)
            binary_images_per_folder.append(binary_image)
            enhanced_binary_images_per_folder.append(enhanced_binary_image)
            contours_images_per_folder.append(contours_image)
            count += 1
    edge_detection_results.append(edge_detection_results_per_folder)
    binary_images.append(binary_images_per_folder)
    enhanced_binary_images.append(enhanced_binary_images_per_folder)
    contours_images.append(contours_images_per_folder)

In [69]:
image_width, image_height = edge_detection_results[0][0].size

In [70]:
grid_image_edges = Image.new('L', (image_width * grid_size[1], image_height * grid_size[0]))
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        folder_index = j
        image_index = i
        if folder_index < len(edge_detection_results) and image_index < len(edge_detection_results[folder_index]):
            img = edge_detection_results[folder_index][image_index]
            grid_image_edges.paste(img, (j * image_width, i * image_height))

In [71]:
grid_image_edges.save(os.path.join(output_dir, "5.Combined_edge_detection_results.png"))

In [72]:
grid_image_binary = Image.new('L', (image_width * grid_size[1], image_height * grid_size[0]))
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        folder_index = j
        image_index = i
        if folder_index < len(binary_images) and image_index < len(binary_images[folder_index]):
            img = binary_images[folder_index][image_index]
            grid_image_binary.paste(img, (j * image_width, i * image_height))

In [73]:
grid_image_binary.save(os.path.join(output_dir, "6.Combined_binary_images.png"))

In [74]:
grid_image_enhanced_binary = Image.new('L', (image_width * grid_size[1], image_height * grid_size[0]))
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        folder_index = j
        image_index = i
        if folder_index < len(enhanced_binary_images) and image_index < len(enhanced_binary_images[folder_index]):
            img = enhanced_binary_images[folder_index][image_index]
            grid_image_enhanced_binary.paste(img, (j * image_width, i * image_height))

In [75]:
grid_image_enhanced_binary.save(os.path.join(output_dir, "7.Combined_enhanced_binary_images.png"))

In [76]:
combined_contours_image = Image.new('RGB', (image_width * grid_size[1], image_height * grid_size[0]))
for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        folder_index = j
        image_index = i
        if folder_index < len(contours_images) and image_index < len(contours_images[folder_index]):
            img = contours_images[folder_index][image_index]
            combined_contours_image.paste(img, (j * image_width, i * image_height))

In [77]:
combined_contours_image.save(os.path.join(output_dir, "8.Combined_contours.png"))

In [78]:
image_filenames = [
    "1.Combined_image.png",
    "2.Combined_image_with_keypoints.png",
    "3.Combined_edge_detection_results.png",
    "4.Combined_lines_visualized_images.png",
    "5.Combined_edge_detection_results.png",
    "6.Combined_binary_images.png",
    "7.Combined_enhanced_binary_images.png",
    "8.Combined_contours.png"
]

In [79]:
frame_duration = 0.2

In [80]:
gif_path = os.path.join(output_dir, "Total.gif")

In [81]:
with imageio.get_writer(gif_path, mode='I', duration=frame_duration) as writer:
    for filename in image_filenames:
        image_path = os.path.join(output_dir, filename)
        image = imageio.imread(image_path)
        writer.append_data(image)

/var/folders/vw/c2j4rkjd52z6m0m84r33nr7r0000gn/T/ipykernel_84367/1825934575.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)
